In [261]:
import pandas as pd

Outline of analysis:

       I: I used the complete spreadsheet of report summaries for campaign finance reports filed by all types of campaign finance committees for 2020, and attached it to a spreadsheet with information about all of the committees.
       
           A. I used that to count the number of a specific type of committee -- political action -- that filed reports that year.
           
           B. I did the same thing for every year, going as far back as 2011.
           
    II: I focused on the reports for specific report deadlines in 2020 (30 days after the Nov. election and the October quarterly reports)
    
           A. I calculated the total money being reported for that election.
           
           B. For the most recent report, and for October reports 2011-2020, I calculated how much money each committee type raised.
           
    III: I looked at the spreadsheet of all discrete donations big enough to report.
    
           A. I sorted the spreadsheet to find the 10 biggest donations. 
           
           B. I calculated:
           
               1. How much money came from each state.
               
               2. Which committees brought in the highest total of itemized donations
               
          C. I repeated many of the 'B.' steps for donations to specific types of committees (PACs, all committees except PACs, candidate committees, etc. 
               
               1. For political action committees, I also:
               
                   a. Repeated the "A." step.
                   
                   b. Calculated which PACs received the most money total. 
               
    IV: I looked at the spreadsheet of expenditures by campaign finance committees that exceeded a specific amount
    
        A. I calculated how many times specific purposes were listed for expenditure and how much money was devoted to each expenditure category
        
        B. I calculated which committees spent the most
        
        C. For political action committees, I calculated
        
           1. How much PACs spent in total.
           
           2. How many times specific purposes were listed for expenditures.
           
           3. Which PACs spent the money money.
           
           3. How much money was spent in each state.
               

I borrowed the list of report summary data types from our earlier analysis in class to bring in the 2020 report summaries csv. I'm labeling the csvs with the date of download to distinguish them.

In [262]:
report_summary_data_types = {
    "MECID": "string",
    "Committee Name": "string",
    "Report": "string",
    "Report Year": "Int64",
    "Report Type": "string",
    "Previous Receipts": "float",
    "Contributions Received": "float",
    "Loans Received": "float",
    " Misc. Receipts": "float",
    " Receipts Subtotal": "float",
    "In-Kind Contributions": "float",
    " Total Receipts This Election": "float",
    "Previous Expenditures": "float",
    "Cash or Check Expenditures": "float",
    "In-Kind Expenditures": "float",
    "Credit Expenditures": "float",
    "Expenditure Subtotal": "float",
    "Total Expenditures": "float",
    "Previous Contributions": "float",
    "Cash/Check Contributions": "float",
    "Credit Contributions": "float",
    "In-Kind Contributions1": "float",
    "Contribution Subtotal": "float",
    "Total Contributions": "float",
    "Loan Disbursements": "float",
    "Disbursements Payments": "float",
    "Misc. Disbursements": "float",
    "Total Disbursements": "float",
    "Starting Money on Hand": "float",
    " Monetary Receipts": "float",
    "Check Disbursements": "float",
    "Cash Disbursements": "float",
    "Total Monetary Disbursements": "float",
    "Ending Money on Hand": "float",
    "Outstanding Indebtedness": "float",
    "Loans Recieved": "float",
    "New Expenditures": "float",
    "New Contributions": "float",
    "Payments Made on Loan": "float",
    "Debt Forgiven on Loans": "float",
    "Total Indebtendness": "float",
}

In [263]:
report_summaries_2020 = pd.read_csv("data/report-summaries/2020/12_4_2020.csv", dtype=report_summary_data_types)

Also borrowing the code to clean up the columns...

In [264]:
def clean_columns(year):
    year.columns = year.columns \
    .str.strip() \
    .str.lower() \
    .str.replace(' ', '_') \
    .str.replace('-', '_') \
    .str.replace('.', '') \
    .str.replace('/', '_')

In [265]:
clean_columns(report_summaries_2020)

Checking to make sure it worked...

In [266]:
report_summaries_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16240 entries, 0 to 16239
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   mecid                         16240 non-null  string 
 1   committee_name                16240 non-null  string 
 2   report                        16240 non-null  string 
 3   report_year                   16240 non-null  Int64  
 4   report_type                   16240 non-null  string 
 5   previous_receipts             12518 non-null  float64
 6   contributions_received        12518 non-null  float64
 7   loans_received                12518 non-null  float64
 8   misc_receipts                 12518 non-null  float64
 9   receipts_subtotal             12518 non-null  float64
 10  in_kind_contributions         12518 non-null  float64
 11  total_receipts_this_election  12518 non-null  float64
 12  previous_expenditures         12518 non-null  float64
 13  c

Here I'm pulling in the list of committees that Craig Hollis from MECID provided. A lot of my analysis will involve sorting by committee type, so this will be essential. Again I'm setting the data types and cleaning up the column names. 

In [267]:
committees_data_types = {
    'MECID' : "string",
    'CommitteeName': "string",
    'Comm_Type' : "string",
    'Comm_Status' : "string",
    'Candidate FName' : "string",
    'Candidate MName' : "string",
    'Candidate LName' : "string",
    'ElectionDate' : "string",
    'Office Sought' : "string",
}

In [268]:
committees = pd.read_csv('data/committee-list/updated.csv') \
    .astype(committees_data_types) \
    .fillna("")

In [269]:
clean_columns(committees)

In [270]:
committees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23209 entries, 0 to 23208
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   mecid            23209 non-null  string
 1   committeename    23209 non-null  string
 2   comm_type        23209 non-null  string
 3   comm_status      23209 non-null  string
 4   candidate_fname  23209 non-null  string
 5   candidate_mname  23209 non-null  string
 6   candidate_lname  23209 non-null  string
 7   electiondate     23209 non-null  string
 8   office_sought    23209 non-null  string
dtypes: string(9)
memory usage: 1.6 MB


I check what my options for committee types are. I'm most interested in PACs so I define a version of the committee list that includes only those with "Political Action" as the committee type.

In [271]:
sorted(committees.comm_type.unique())

['Campaign',
 'Candidate',
 'Debt Service',
 'Exemption',
 'Exploratory',
 'Political Action',
 'Political Party']

In [272]:
pacs = committees[committees.comm_type == 'Political Action'].reset_index(drop=True)

Now I merge that committee list with the 2020 report summaries, using the unique committee identifier (mecid). However, I can't validate one-to-many. It appears the identifier might not actually be unique. I look at the csv file, which I had open in Excel, show that that's the case.

If I count how many values there are for each MECID, the length of the list should give me the total number of MECIDs, and therefore the number of unique PACs that filed reports in 2020. 

In [273]:
def count_pacs(report_summaries_year):
    pacs_year = pacs.merge(report_summaries_year, on='mecid',).reset_index(drop=True)
    return len(pacs_year.mecid.value_counts())

In [274]:
count_pacs(report_summaries_2020)

1035

I'd like to know how the number of PACs filing reports each year has changed, so I'm just going to repeat this process over and over, pulling in the csv from each year all the way back to 2020.

In [275]:
report_summaries_2019 = pd.read_csv("data/report-summaries/2019/11_1_2020.csv", dtype=report_summary_data_types)

In [276]:
clean_columns(report_summaries_2019)

In [277]:
count_pacs(report_summaries_2019)

1006

In [278]:
report_summaries_2018 = pd.read_csv("data/report-summaries/2018/11_1_2020.csv", dtype=report_summary_data_types)

In [279]:
clean_columns(report_summaries_2018)

In [280]:
count_pacs(report_summaries_2018)

967

In [281]:
report_summaries_2017 = pd.read_csv("data/report-summaries/2017/11_1_2020.csv", dtype=report_summary_data_types)

In [282]:
clean_columns(report_summaries_2017)

In [283]:
count_pacs(report_summaries_2017)

887

In [284]:
report_summaries_2016 = pd.read_csv("data/report-summaries/2016/11_1_2020.csv", dtype=report_summary_data_types)

In [285]:
clean_columns(report_summaries_2016)

In [286]:
count_pacs(report_summaries_2016)

830

In [287]:
report_summaries_2015 = pd.read_csv("data/report-summaries/2015/11_1_2020.csv", dtype=report_summary_data_types)

In [288]:
clean_columns(report_summaries_2015)

In [289]:
count_pacs(report_summaries_2015)

817

In [290]:
report_summaries_2014 = pd.read_csv("data/report-summaries/2014/11_1_2020.csv", dtype=report_summary_data_types)

In [291]:
clean_columns(report_summaries_2014)

In [292]:
count_pacs(report_summaries_2014)

813

In [293]:
report_summaries_2013 = pd.read_csv("data/report-summaries/2013/11_1_2020.csv", dtype=report_summary_data_types)

In [294]:
clean_columns(report_summaries_2013)

In [295]:
count_pacs(report_summaries_2013)

836

In [296]:
report_summaries_2012 = pd.read_csv("data/report-summaries/2012/11_1_2020.csv", dtype=report_summary_data_types)

In [297]:
clean_columns(report_summaries_2012)

In [298]:
count_pacs(report_summaries_2012)

890

In [299]:
report_summaries_2011 = pd.read_csv("data/report-summaries/2011/11_1_2020.csv", dtype=report_summary_data_types)

In [300]:
clean_columns(report_summaries_2011)

In [301]:
count_pacs(report_summaries_2011)

927

Now I'd like to compare various committee types in the present day. I'm going to focus on the most recent report -- this can easily be modified when a new report comes out.

In [302]:
latest_report_2020 = report_summaries_2020[report_summaries_2020.report == '30 Day After General Election-11/3/2020']

I'd like to know the total that committees reported receiving in the entire election cycle. I'll do this by totaling the "total_receipts_this_election" column for every committee that filed a report 30 days after the general election. I should note, however, that this might be greater than the amount of money that flowed into Missouri politics overall because committees can pass money around, meaning it would be at least double counted. On the other hand, I'm likely missing some committees that didn't file this specific report but still brought in money.

In [303]:
latest_report_2020.total_receipts_this_election.sum()

139180002.58999997

Merging the list of latest report summaries and then checking the total receipts this election confirms the problem with the way my merge is working. It's creating duplicates of reports that are distorting the total. That's because there are multiples of various MECIDs in the committee list. 

In [304]:
latest_committees_2020 = committees \
    .merge(latest_report_2020, on='mecid') \
    .reset_index(drop=True)

In [305]:
latest_committees_2020.total_receipts_this_election.sum()

300437996.23

In [306]:
latest_committees_2020.mecid.value_counts()

C011182    14
C071030    12
C101084    12
C061710    12
C081024    12
           ..
A201449     1
C190963     1
C191052     1
C000537     1
C161298     1
Name: mecid, Length: 626, dtype: Int64

To fix this problem, I worked with James to figure out how to group the committee list by MECID, committee name, type and status.

In [307]:
by_mecid = committees.groupby(by=['mecid', 'committeename', 'comm_type', 'comm_status']) \
.nunique().reset_index()

We tested to make sure that there aren't multiple values for name, type and status with the same MECID. 

In [308]:
test_by_mecid = committees.groupby(by=['mecid']).nunique()

In [309]:
test_by_mecid[test_by_mecid['comm_status']>1]

,committeename,comm_type,comm_status,candidate_fname,candidate_mname,candidate_lname,electiondate,office_sought
mecid,,,,,,,,


Now the merge works better and we're able to validate, then simply group by committee type to get the number of PACs. The result matches what I did before, which gives me confidence that my workaround was valid. But it will allow me to do a lot of analysis that my workaround wouldn't have helped with. 

In [310]:
reports_commtype_2020 = report_summaries_2020.merge(by_mecid, on='mecid', validate='many_to_one')

In [311]:
reports_commtype_2020.groupby(by=['comm_type']).mecid.nunique()

comm_type
Campaign              76
Candidate           1889
Debt Service          24
Exemption             31
Exploratory            2
Political Action    1035
Political Party      226
Name: mecid, dtype: int64

Now I can merge by latest report document with the committee list and figure out how much each committee types has brought in this year. 

In [312]:
latest_report_by_mecid_2020 = latest_report_2020.merge(by_mecid, on='mecid', validate='many_to_one')

In [313]:
latest_report_by_mecid_2020.total_receipts_this_election.sum()

139169402.14

In [314]:
latest_committee_totals = latest_report_by_mecid_2020.groupby(by=['comm_type']).total_receipts_this_election.sum().reset_index()

In [315]:
latest_committee_totals.to_csv('30_day_committee_totals.csv', index = False)

I'll do the same for the October reports.

In [316]:
def get_oct_committee_type_totals(report_summaries_year):
    oct_report_year = report_summaries_year[report_summaries_year.report == 'October Quarterly Report']
    oct_report_by_mecid_year = oct_report_year.merge(by_mecid, on='mecid', validate='many_to_one')
    oct_committee_year = oct_report_by_mecid_year.groupby(by=['comm_type']).total_receipts_this_election.sum().reset_index()
    return oct_committee_year

In [317]:
get_oct_committee_type_totals(report_summaries_2020)

,comm_type,total_receipts_this_election
0,Campaign,4259939.46
1,Candidate,38198375.92
2,Debt Service,944137.62
3,Exemption,0.00
4,Exploratory,857.55
5,Political Action,67290642.88
6,Political Party,3302672.69


In [318]:
get_oct_committee_type_totals(report_summaries_2020).to_csv('oct_2020_committees.csv', index = False)

It seems easier to compare October quarterly reports across different years rather than reports tied to a specific election, since everyone has to file them. So I'll repeat the same process for all the years back to 2011. 

In [319]:
get_oct_committee_type_totals(report_summaries_2019)

,comm_type,total_receipts_this_election
0,Campaign,6990136.16
1,Candidate,49158700.28
2,Debt Service,403990.59
3,Exemption,0.00
4,Exploratory,1178.95
5,Political Action,25962960.09
6,Political Party,1506486.12


In [320]:
get_oct_committee_type_totals(report_summaries_2018)

,comm_type,total_receipts_this_election
0,Campaign,4629676.83
1,Candidate,68047396.05
2,Debt Service,913453.91
3,Exemption,0.00
4,Political Action,34971337.09
5,Political Party,2047529.53


In [321]:
get_oct_committee_type_totals(report_summaries_2017)

,comm_type,total_receipts_this_election
0,Campaign,3219151.87
1,Candidate,52372683.45
2,Debt Service,647633.56
3,Exemption,0.00
4,Political Action,19478111.30
5,Political Party,1521362.35


In [322]:
get_oct_committee_type_totals(report_summaries_2016)

,comm_type,total_receipts_this_election
0,Campaign,8585109.04
1,Candidate,77828766.54
2,Debt Service,9308596.65
3,Exemption,0.00
4,Political Action,40382735.33
5,Political Party,3034716.21


In [323]:
get_oct_committee_type_totals(report_summaries_2015)

,comm_type,total_receipts_this_election
0,Campaign,1453996.56
1,Candidate,27292848.45
2,Debt Service,2685141.55
3,Exemption,0.00
4,Political Action,13931730.45
5,Political Party,1368331.91


In [324]:
get_oct_committee_type_totals(report_summaries_2014)

,comm_type,total_receipts_this_election
0,Campaign,2736554.10
1,Candidate,28286404.82
2,Debt Service,2446955.70
3,Exemption,0.00
4,Political Action,27747251.97
5,Political Party,2401236.89


In [325]:
get_oct_committee_type_totals(report_summaries_2013)

,comm_type,total_receipts_this_election
0,Campaign,1091705.87
1,Candidate,15994329.67
2,Debt Service,3017634.18
3,Political Action,21636718.92
4,Political Party,1408190.20


In [326]:
get_oct_committee_type_totals(report_summaries_2012)

,comm_type,total_receipts_this_election
0,Campaign,8173521.46
1,Candidate,43573104.86
2,Debt Service,5101178.23
3,Exemption,0.00
4,Political Action,19667347.88
5,Political Party,2964294.45


In [327]:
get_oct_committee_type_totals(report_summaries_2011)

,comm_type,total_receipts_this_election
0,Campaign,572741.35
1,Candidate,26609945.81
2,Debt Service,2341331.06
3,Exemption,0.00
4,Exploratory,15759.28
5,Political Action,12162926.50
6,Political Party,1434013.85


Now I want to look at specific donations. The "contributions received" spreadsheet gives information about specific donors/donations over 100 from individuals or any donations from committees. Once again I'll set the data types, import the csv and clean up the column names. 

In [328]:
contributions_received_data_types = {
    "CD1_A ID": "string",
    "MECID": "string",
    "Committee Name": "string",
    "Committee": "string",
    "Company": "string",
    "First Name": "string",
    "Last Name": "string",
    "Address 1": "string",
    "Address 2": "string",
    "City": "string",
    "State": "string",
    "Zip": "string",
    "Employer": "string",
    "Occupation": "string",
    "Date": "string",
    "Amount": "float",
    "Contribution Type": "string",
    "Report": "string",
}
    
    
    
    
    
    
    

In [329]:
received_2020 = pd.read_csv("data/contributions-received/12_4_2020.csv", dtype=contributions_received_data_types)

In [330]:
clean_columns(received_2020)

Sorting to determine the biggest individual donations...

In [331]:
received_2020.sort_values('amount', ascending=False).head(10)

,cd1_a_id,mecid,committee_name,committee,company,first_name,last_name,address_1,address_2,city,state,zip,employer,occupation,date,amount,contribution_type,report
195296,1531035,C201214,Republican Governors Association - Missouri 2020,<NA>,Republican Governors Association,<NA>,<NA>,"1747 Pennsylvania Avenue, NW",Suite 250,Washington,DC,20006,<NA>,<NA>,10/8/2020 12:00:00 AM,4000000.0,M,8 Day Before General Election-11/3/2020
189385,1523919,C180490,Uniting Missouri PAC,REPUBLICAN GOVERNORS ASSOCIATION - MISSOURI 2020,<NA>,<NA>,<NA>,1747 PENNSYLVANIA AVE NW,SUITE 250,WASHINGTON,DC,20006,<NA>,<NA>,10/9/2020 12:00:00 AM,3000000.0,M,8 Day Before General Election-11/3/2020
189358,1523892,C180490,Uniting Missouri PAC,REPUBLICAN GOVERNORS ASSOCIATION - MISSOURI 2020,<NA>,<NA>,<NA>,1747 PENNSYLVANIA AVE NW,SUITE 250,WASHINGTON,DC,20006,<NA>,<NA>,10/1/2020 12:00:00 AM,3000000.0,M,8 Day Before General Election-11/3/2020
139108,1464448,C201214,Republican Governors Association - Missouri 2020,<NA>,Republican Governors Association,<NA>,<NA>,"1747 Pennsylvania Avenue, NW",Suite 250,Washington,DC,20006,<NA>,<NA>,8/3/2020 12:00:00 AM,2500000.0,M,30 Day After Primary Election-8/4/2020
156571,1488932,C201214,Republican Governors Association - Missouri 2020,<NA>,Republican Governors Association,<NA>,<NA>,"1747 Pennsylvania Avenue, NW",Suite 250,Washington,DC,20006,<NA>,<NA>,9/30/2020 12:00:00 AM,2500000.0,M,October Quarterly Report
156567,1488928,C201214,Republican Governors Association - Missouri 2020,<NA>,Republican Governors Association,<NA>,<NA>,"1747 Pennsylvania Avenue, NW",Suite 250,Washington,DC,20006,<NA>,<NA>,8/31/2020 12:00:00 AM,2500000.0,M,October Quarterly Report
174060,1508342,C180490,Uniting Missouri PAC,REPUBLICAN GOVERNORS ASSOCIATION - MISSOURI 2020,<NA>,<NA>,<NA>,1747 PENNSYLVANIA AVE NW,SUITE 250,WASHINGTON,DC,20006,<NA>,<NA>,9/15/2020 12:00:00 AM,2000000.0,M,October Quarterly Report
139010,1464167,C180490,Uniting Missouri PAC,REPUBLICAN GOVERNORS ASSOCIATION - MISSOURI 2020,<NA>,<NA>,<NA>,1747 PENNSYLVANIA AVE NW,SUITE 250,WASHINGTON,DC,20006,<NA>,<NA>,8/5/2020 12:00:00 AM,2000000.0,M,30 Day After Primary Election-8/4/2020
205501,1541565,C180490,Uniting Missouri PAC,REPUBLICAN GOVERNORS ASSOCIATION - MISSOURI 2020,<NA>,<NA>,<NA>,1747 PENNSYLVANIA AVE NW,SUITE 250,WASHINGTON,DC,20006,<NA>,<NA>,10/26/2020 12:00:00 AM,1700000.0,M,24 Hour Expenditure Report-11/3/2020 General
205603,1541677,C201214,Republican Governors Association - Missouri 2020,<NA>,Republican Governors Association,<NA>,<NA>,"1747 Pennsylvania Avenue, NW",Suite 250,Washington,DC,20006,<NA>,<NA>,10/23/2020 12:00:00 AM,1600000.0,M,24 Hour Expenditure Report-11/3/2020 General


...and sending that data to a csv file.

In [332]:
received_2020.sort_values('amount', ascending=False).head(40).to_csv('contributions_received_2020.csv', index=False)

I want to group donations by state to find out where most of the donations are coming from. Unfortunately, there are more than the 51 rows I expected. Making sure all the states are uppercase helps a little. There are still some fake states but I don't think any of them gave enough money to throw off my conclusions. 

In [333]:
received_2020.state = received_2020.state.str.upper()

In [334]:
states = received_2020.groupby(['state']).amount.sum().reset_index()

In [335]:
states.to_csv('states_dec.csv', index=False) 

I merged the received spreasheet with committee types, first looking at PACs specifically.

In [336]:
received_commtype_2020 = received_2020.merge(by_mecid, on='mecid', validate='many_to_one')

In [337]:
committees = received_commtype_2020.groupby(['comm_type','committee_name']).amount.sum().reset_index()

In [338]:
highest_committees = committees.sort_values('amount', ascending=False).reset_index().head(10)

In [339]:
highest_committees.to_csv('highest_committees.csv', index=False)

In [340]:
pacs_received_2020 = received_commtype_2020[received_commtype_2020.comm_type == "Political Action"]

In [341]:
pacs_received_2020.reset_index().head(3)

,index,cd1_a_id,mecid,committee_name,committee,company,first_name,last_name,address_1,address_2,...,contribution_type,report,committeename,comm_type,comm_status,candidate_fname,candidate_mname,candidate_lname,electiondate,office_sought
0,1894,1257094,C151190,Missouri Alliance PAC,<NA>,Kiel Center Partners LP,<NA>,<NA>,1401 Clark Ave.,13447 Land O woods Dr.,...,M,January Quarterly Report,Missouri Alliance PAC,Political Action,Active,1,1,1,1,1
1,1895,1257095,C151190,Missouri Alliance PAC,<NA>,Comcast Financial Agency Corporation,<NA>,<NA>,1701 JFK Blvd,<NA>,...,M,January Quarterly Report,Missouri Alliance PAC,Political Action,Active,1,1,1,1,1
2,1896,1369903,C151190,Missouri Alliance PAC,<NA>,Anesthesia Associates of Kansas City PC,<NA>,<NA>,8717 W. 110th St.,<NA>,...,M,April Quarterly Report,Missouri Alliance PAC,Political Action,Active,1,1,1,1,1


In [342]:
biggest_pac_receivers = pacs_received_2020.groupby(by=['committee_name']).amount.sum().reset_index()

In [343]:
biggest_pac_receivers.sort_values('amount', ascending=False).reset_index().head(10)

,index,committee_name,amount
0,642,Uniting Missouri PAC,18527737.37
1,527,Republican Governors Association - Missouri 2020,14850207.28
2,19,A Stronger Missouri,13771222.55
3,127,Democratic Governors Association - Missouri,9022789.55
4,72,CLEAN Missouri,7346869.04
5,256,Keep Government Accountable,2743373.14
6,415,Missouri Forward PAC,2305094.03
7,212,"House Republican Campaign Committee, Inc",2255210.91
8,358,MO Opportunity PAC,1668604.01
9,429,Missouri Senate Campaign Committee,1607977.00


This list isn't too different than what I had before, since PACs received the biggest donations. 

In [344]:
pacs_received_2020.sort_values('amount', ascending=False).head(5)

,cd1_a_id,mecid,committee_name,committee,company,first_name,last_name,address_1,address_2,city,...,contribution_type,report,committeename,comm_type,comm_status,candidate_fname,candidate_mname,candidate_lname,electiondate,office_sought
212153,1531035,C201214,Republican Governors Association - Missouri 2020,<NA>,Republican Governors Association,<NA>,<NA>,"1747 Pennsylvania Avenue, NW",Suite 250,Washington,...,M,8 Day Before General Election-11/3/2020,Republican Governors Association - Missouri 2020,Political Action,Active,1,1,1,1,1
147170,1523919,C180490,Uniting Missouri PAC,REPUBLICAN GOVERNORS ASSOCIATION - MISSOURI 2020,<NA>,<NA>,<NA>,1747 PENNSYLVANIA AVE NW,SUITE 250,WASHINGTON,...,M,8 Day Before General Election-11/3/2020,Uniting Missouri PAC,Political Action,Active,1,1,1,1,1
147143,1523892,C180490,Uniting Missouri PAC,REPUBLICAN GOVERNORS ASSOCIATION - MISSOURI 2020,<NA>,<NA>,<NA>,1747 PENNSYLVANIA AVE NW,SUITE 250,WASHINGTON,...,M,8 Day Before General Election-11/3/2020,Uniting Missouri PAC,Political Action,Active,1,1,1,1,1
212151,1488932,C201214,Republican Governors Association - Missouri 2020,<NA>,Republican Governors Association,<NA>,<NA>,"1747 Pennsylvania Avenue, NW",Suite 250,Washington,...,M,October Quarterly Report,Republican Governors Association - Missouri 2020,Political Action,Active,1,1,1,1,1
212150,1488928,C201214,Republican Governors Association - Missouri 2020,<NA>,Republican Governors Association,<NA>,<NA>,"1747 Pennsylvania Avenue, NW",Suite 250,Washington,...,M,October Quarterly Report,Republican Governors Association - Missouri 2020,Political Action,Active,1,1,1,1,1


Also grouping them by state and donation size. 2,650 is the contribution limit for statewide office, which PACS don't have to follow.

In [345]:
state_pacs = pacs_received_2020.groupby(['state']).amount.sum().reset_index()

In [346]:
state_pacs.to_csv('state_pacs_dec.csv', index=False) 

Repeating some of the same steps with everything except PACs...

In [347]:
no_pacs_received_2020 = received_commtype_2020[received_commtype_2020.comm_type != 'Political Action']

In [348]:
no_pacs_received_2020.head(3)

,cd1_a_id,mecid,committee_name,committee,company,first_name,last_name,address_1,address_2,city,...,contribution_type,report,committeename,comm_type,comm_status,candidate_fname,candidate_mname,candidate_lname,electiondate,office_sought
0,1256570,A151115,Friends Of Wesley Bell,<NA>,<NA>,Jerome,Husgen,200 Cabellero Drive,<NA>,Ballwin,...,M,January Quarterly Report,Friends Of Wesley Bell,Candidate,Active,1,1,1,4,2
1,1256571,A151115,Friends Of Wesley Bell,<NA>,<NA>,Max,Micallef,735 Anderson Hill Road,<NA>,Purchase,...,M,January Quarterly Report,Friends Of Wesley Bell,Candidate,Active,1,1,1,4,2
2,1256572,A151115,Friends Of Wesley Bell,<NA>,<NA>,Duncan,Smith,2342 Granite Way,<NA>,Sacramento,...,M,January Quarterly Report,Friends Of Wesley Bell,Candidate,Active,1,1,1,4,2


In [349]:
no_pacs_received_2020.sort_values('amount', ascending=False).head(3)

,cd1_a_id,mecid,committee_name,committee,company,first_name,last_name,address_1,address_2,city,...,contribution_type,report,committeename,comm_type,comm_status,candidate_fname,candidate_mname,candidate_lname,electiondate,office_sought
38919,1440204,C190918,Missourians for Healthcare,<NA>,North Fund,<NA>,<NA>,1101 Connecticut Ave NW Suite 450,<NA>,Washington,...,M,8 Day Before Primary Election-8/4/2020,Missourians for Healthcare,Campaign,Active,1,1,1,1,1
38865,1402368,C190918,Missourians for Healthcare,<NA>,Health Forward Foundation,<NA>,<NA>,2300 Main Street Suite 304,<NA>,Kansas City,...,M,July Quarterly Report,Missourians for Healthcare,Campaign,Active,1,1,1,1,1
38926,1440211,C190918,Missourians for Healthcare,<NA>,Missouri Hospital Association,<NA>,<NA>,PO Box 60,<NA>,Jefferson City,...,M,8 Day Before Primary Election-8/4/2020,Missourians for Healthcare,Campaign,Active,1,1,1,1,1


In [350]:
no_pacs_received_2020.groupby(['state']).amount.sum()

state
AE      940.00
AK     3640.40
AL    14424.70
AP      213.50
AR    38482.95
        ...   
WA    51871.22
WI     8786.78
WV     3060.10
WY     1427.70
XX      110.00
Name: amount, Length: 65, dtype: float64

...then with candidate committees. I'm less interested in small vs. large donations for candidates because many of them have limits anyway. 

In [351]:
candidates_received_2020 = received_commtype_2020[received_commtype_2020.comm_type == 'Candidate']

In [352]:
candidates_received_2020.head(5)

,cd1_a_id,mecid,committee_name,committee,company,first_name,last_name,address_1,address_2,city,...,contribution_type,report,committeename,comm_type,comm_status,candidate_fname,candidate_mname,candidate_lname,electiondate,office_sought
0,1256570,A151115,Friends Of Wesley Bell,<NA>,<NA>,Jerome,Husgen,200 Cabellero Drive,<NA>,Ballwin,...,M,January Quarterly Report,Friends Of Wesley Bell,Candidate,Active,1,1,1,4,2
1,1256571,A151115,Friends Of Wesley Bell,<NA>,<NA>,Max,Micallef,735 Anderson Hill Road,<NA>,Purchase,...,M,January Quarterly Report,Friends Of Wesley Bell,Candidate,Active,1,1,1,4,2
2,1256572,A151115,Friends Of Wesley Bell,<NA>,<NA>,Duncan,Smith,2342 Granite Way,<NA>,Sacramento,...,M,January Quarterly Report,Friends Of Wesley Bell,Candidate,Active,1,1,1,4,2
3,1256573,A151115,Friends Of Wesley Bell,<NA>,<NA>,Vanita,Mishra,29 Woodlawn Drive,<NA>,New Ulm,...,M,January Quarterly Report,Friends Of Wesley Bell,Candidate,Active,1,1,1,4,2
4,1257056,A151115,Friends Of Wesley Bell,<NA>,<NA>,Christopher,Meyers,7617 Jerome Avenue,<NA>,Maplewood,...,M,January Quarterly Report,Friends Of Wesley Bell,Candidate,Active,1,1,1,4,2


In [353]:
candidates_received_2020.sort_values('amount', ascending=False).head(5)

,cd1_a_id,mecid,committee_name,committee,company,first_name,last_name,address_1,address_2,city,...,contribution_type,report,committeename,comm_type,comm_status,candidate_fname,candidate_mname,candidate_lname,electiondate,office_sought
209645,1436970,C121334,Citizens To Elect Kimberly Gardner,Missouri Justice & Public Safety PAC,<NA>,<NA>,<NA>,700 13th St. NW Ste 800,<NA>,Washington,...,I,July Quarterly Report,Citizens To Elect Kimberly Gardner,Candidate,Active,1,1,1,8,2
29144,1450029,C191045,Terry Dotson for Cass County Sheriff,<NA>,<NA>,Robert,Cutler,10955 Granada Lane,<NA>,Overland Park,...,M,8 Day Before Primary Election-8/4/2020,Terry Dotson for Cass County Sheriff,Candidate,Active,1,1,1,1,1
201045,1524117,C191065,David Lenihan for Senate.com,<NA>,<NA>,David,Lenihan,19 Kings Crossing Ct,<NA>,Saint Louis,...,I,8 Day Before General Election-11/3/2020,David Lenihan for Senate.com,Candidate,Active,1,1,1,2,1
37601,1505113,C190913,Michelle Sherod for St. Louis,<NA>,<NA>,Michelle,Sherod,4005 McRee,4005 McRee,St Louis,...,I,Termination,Michelle Sherod for St. Louis,Candidate,Active,1,1,1,1,1
217245,1472144,C201154,Friends of Jeffrey Boyd,Citizens to Elect Jeffrey L Boyd,<NA>,<NA>,<NA>,5879 Dr Martin Luther King Dr,<NA>,St Louis,...,M,30 Day After Primary Election-8/4/2020,Friends of Jeffrey Boyd,Candidate,Terminated,1,1,1,1,1


In [354]:
state_candidates = candidates_received_2020.groupby(['state']).amount.sum().reset_index()

In [355]:
state_candidates.to_csv('state_candidates_dec.csv', index=False)

What can I learn about expenditures? I have to bring in yet another csv, same steps as before. Also merging with the committee list again, comparing by committee type and filtering for PACs. 

In [356]:
expenditures_data_types = {
    "CD3_B ID": "string",
    "MECID": "string",
    "Committee Name": "string",
    "First Name": "string",
    "Last Name": "string",
    "Company": "string",
    "Address 1": "string",
    "Address 2": "string",
    "City": "string",
    "State": "string",
    "Zip": "string",
    "Date": "string",
    "Purpose": "string",
    "Amount": "float",
    "Expenditure Type": "string",
    "Report": "string",
}
    
    
    

In [357]:
expenditures_2020 = pd.read_csv("data/expenditures/11_23_2020.csv", dtype=expenditures_data_types)

In [358]:
clean_columns(expenditures_2020)

In [359]:
expenditures_2020

,cd3_b_id,mecid,committee_name,first_name,last_name,company,address_1,address_2,city,state,zip,date,purpose,amount,expenditure_type,report
0,379044,A151115,Friends Of Wesley Bell,<NA>,<NA>,CBTU,PO BOX 5034,<NA>,St Louis,MO,63115,10/15/2019 12:00:00 AM,Donation,180.00,P,January Quarterly Report
1,379045,A151115,Friends Of Wesley Bell,<NA>,<NA>,Maconite Scholarship,PO BOX 211446,<NA>,St Louis,MO,63121,10/15/2019 12:00:00 AM,Donation,100.00,P,January Quarterly Report
2,379114,C151126,Roberts For St Louis,<NA>,<NA>,Missouri Democrat Party,4218 Roanoke Road #304,<NA>,Kansas City,MO,64111,10/7/2019 12:00:00 AM,VAN Access,1000.00,P,January Quarterly Report
3,379115,C151126,Roberts For St Louis,<NA>,<NA>,US Post Office,1720 Market,<NA>,St Louis,MO,63155,10/8/2019 12:00:00 AM,Postage,55.00,P,January Quarterly Report
4,379116,C151126,Roberts For St Louis,<NA>,<NA>,Mark's Quick Print,9567 Page Avenue,<NA>,St Louis,MO,63132,10/18/2019 12:00:00 AM,Printing,102.29,P,January Quarterly Report
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35328,422970,A201368,Kater For Commissioner 2020,<NA>,R GILLIAM REAL ESTATE LLC,"Richard Black, CPA, PC",PO Box 211,<NA>,Farmington,MO,63640,11/13/2020 12:00:00 AM,Accounting,434.78,P,30 Day After General Election-11/3/2020
35329,422971,A201368,Kater For Commissioner 2020,<NA>,<NA>,KFMO-B104,804 St Joe Drive,804 St Joe Drive,Park Hills,MO,63601,10/31/2020 12:00:00 AM,Advertising,390.00,P,30 Day After General Election-11/3/2020
35330,422982,A201587,Committee to Elect Steve Atkinson,<NA>,<NA>,sams club,101 conley rd,<NA>,columbia,MO,65201,10/28/2020 12:00:00 AM,candy for parade,197.65,P,Termination
35331,422983,A201587,Committee to Elect Steve Atkinson,<NA>,<NA>,party city,broadway,<NA>,columbia,MO,65201,10/28/2020 12:00:00 AM,candy decorations,106.60,P,Termination


In [360]:
expenditures_commtype_2020 = expenditures_2020.merge(by_mecid, on='mecid', validate='many_to_one')

In [361]:
committees = expenditures_commtype_2020.groupby(['comm_type','committee_name']).amount.sum().reset_index()

In [362]:
highest_spenders = committees.sort_values('amount', ascending=False).reset_index().head(10)

In [363]:
highest_spenders.to_csv('top_spenders.csv', index=False)

In [364]:
expenditures_commtype_2020.purpose = expenditures_commtype_2020.purpose.str.lower()

In [365]:
expenditure_purpose = expenditures_commtype_2020.groupby(by=['purpose']).amount.sum().reset_index()

In [366]:
top_expenditure_purpose = expenditure_purpose.sort_values('amount', ascending=False)

In [367]:
top_expenditure_purpose

,purpose,amount
4552,media buy,19812048.66
417,advertising - media placement,14540244.74
402,advertising,6448780.34
4556,media buys,5093093.72
2333,direct mail,2334806.85
...,...,...
3954,kessel donation processing fee,0.53
8152,wu donation processing fee,0.53
5208,payroll error,0.40
7473,test transaction,0.01


In [368]:
top_expenditure_purpose.to_csv("expenditure_purpose.csv")

In [369]:
expenditures_commtype_2020.groupby(by=['comm_type']).amount.sum()

comm_type
Campaign             9926209.80
Candidate           38013182.25
Debt Service          947978.82
Political Action    66312029.66
Political Party      2505314.68
Name: amount, dtype: float64

In [370]:
pacs_expenditures_2020 = expenditures_commtype_2020[expenditures_commtype_2020.comm_type == "Political Action"]

In [371]:
pacs_expenditures_2020.reset_index()

,index,cd3_b_id,mecid,committee_name,first_name,last_name,company,address_1,address_2,city,...,expenditure_type,report,committeename,comm_type,comm_status,candidate_fname,candidate_mname,candidate_lname,electiondate,office_sought
0,871,379800,C101515,Lincoln County Democrat Club,<NA>,<NA>,Lincoln County Fairgrounds,#1 Fairgrounds Circle,<NA>,Troy,...,P,January Quarterly Report,Lincoln County Democrat Club,Political Action,Active,1,1,1,1,1
1,872,379801,C101515,Lincoln County Democrat Club,<NA>,<NA>,Elsberry FFA,138,Tomahawk Dr,Elsberry,...,P,January Quarterly Report,Lincoln County Democrat Club,Political Action,Active,1,1,1,1,1
2,873,379802,C101515,Lincoln County Democrat Club,<NA>,<NA>,Buzzy Liquor,208 North Lincon Dr,<NA>,Troy,...,P,January Quarterly Report,Lincoln County Democrat Club,Political Action,Active,1,1,1,1,1
3,874,379803,C101515,Lincoln County Democrat Club,<NA>,<NA>,One Call Now,6450,POE Avenue Suite 500,Dayton,...,P,January Quarterly Report,Lincoln County Democrat Club,Political Action,Active,1,1,1,1,1
4,875,379887,C101515,Lincoln County Democrat Club,<NA>,<NA>,Lincoln County Fairgrounds,#1 Fairgrounds Circle,<NA>,Troy,...,P,January Quarterly Report,Lincoln County Democrat Club,Political Action,Active,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,35297,421951,C201410,Northwest Missouri Leadership PAC,<NA>,<NA>,MO-PAC Compliance,PO Box 52,<NA>,Jefferson City,...,P,8 Day Before General Election-11/3/2020,Northwest Missouri Leadership PAC,Political Action,Active,1,1,1,1,1
6008,35303,422330,C171160,JCDC Truman PAC,Patrick,Shami,TJP Strategies,1124 NE Contry Ln,<NA>,Lees Summit,...,I,24 Hour Expenditure Report-11/3/2020 General,JCDC Truman PAC,Political Action,Active,1,1,1,1,1
6009,35306,422372,C201476,Missourians for Fair Courts,<NA>,<NA>,Milman Research & Consulting LLC,14716 Top Sergeant Lane,<NA>,Centreville,...,P,24 Hour Expenditure Report-11/3/2020 General,Missourians for Fair Courts,Political Action,Active,1,1,1,1,1
6010,35309,422381,C010081,MO CHAMBER PAC,<NA>,<NA>,DDI,8315 Drury Industrial Parkway,<NA>,St Louis,...,P,24 Hour Expenditure Report-11/3/2020 General,MO Chamber PAC,Political Action,Active,1,1,1,1,1


These seem the hardest to analyze in a meaningful way. Grouping by purpose, there are 1739 different purposes listed for spending by PACs. 

In [372]:
pacs_expenditures_2020.purpose.value_counts()

campaign worker                                  443
accounting fees                                  132
advertising                                      132
consulting                                       107
printing                                         100
                                                ... 
due cards                                          1
payment for stubhub charge for parking passes      1
adminstrative fee                                  1
party city - event supplies                        1
design work - endorsement ads                      1
Name: purpose, Length: 1739, dtype: Int64

In [373]:
pac_expenditures_purpose = pacs_expenditures_2020.groupby(by=['purpose']).amount.sum().reset_index()

In [374]:
top_pac_expenditures = pac_expenditures_purpose.sort_values('amount', ascending=False)

In [375]:
top_pac_expenditures

,purpose,amount
920,media buy,15375634.00
95,advertising - media placement,14540244.74
91,advertising,4200694.16
919,media advertising,2284983.00
439,direct mail,1844355.37
...,...,...
1049,owed taxes,3.00
999,newspaper subscription,3.00
872,mailing supplies,1.09
373,copy of deposited checks,1.00


In [376]:
top_pac_expenditures.to_csv("pac_expenditures.csv", index=False)

In [377]:
pacs_expenditures_2020.purpose.value_counts().to_csv("pac.csv", index=False)

In [378]:
pacs_expenditures_2020.amount.sum()

66312029.66

These are the PACs that spent the most money. 

In [379]:
biggest_pac_spenders = pacs_expenditures_2020.groupby(by=['committee_name']).amount.sum().reset_index()

In [380]:
biggest_pac_spenders.sort_values('amount', ascending=False).head(10)

,committee_name,amount
496,Uniting Missouri PAC,22313214.16
18,A Stronger Missouri,11718712.05
59,CLEAN Missouri,5161579.37
284,MO Opportunity PAC,2624548.94
178,"House Republican Campaign Committee, Inc",2285581.02
321,Missouri Forward PAC,1580378.19
379,Page PAC,1385192.66
31,American Dream PAC,1142313.53
391,Protect Missouri Freedom,1120138.49
177,House Democratic Campaign Committee,1081504.57


Where do PACs spend their money?

In [381]:
pac_spending_by_state = pacs_expenditures_2020.groupby(by=['state']).amount.sum().reset_index()

In [382]:
pac_spending_by_state.sort_values('amount', ascending=False)

,state,amount
38,VA,19450193.08
21,MO,13938021.79
7,DC,13795759.50
11,IA,6412389.29
9,FL,5855530.63
12,IL,2251263.05
3,AZ,1038654.70
29,OH,861752.15
36,TX,653839.02
4,CA,386325.00
